Каспаров Иоанн 6132


Лабораторная работа 2


Сумма элементов вектора




In [ ]:
!/usr/local/cuda/bin/nvcc --version
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin
!cuda-install-samples-11.2.sh ~ && cd /root/NVIDIA_CUDA-11.2_Samples

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-dr5dqjt3
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-dr5dqjt3
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0d2ab99cccbbc682722e708515fe9c4cfc50185a
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4716 sha256=561d29a549146033a5f6c072b3d93b46accd4ad2c33eae8b2194a2709ea2149b
  Stored in directory: /tmp/pip-ephem-wheel-cache-nrf59cd4/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out
/bin/bash: line 1: cuda-install-

In [ ]:
%%cu

#include <cstdlib>
#include <curand.h>
#include <cuda_runtime.h>
#include <cublas_v2.h>
#include <malloc.h>
#include <stdio.h>
#include <stdlib.h>
#include <iostream>


__global__ void addKernel(int* c, int* a, int* b, unsigned int size)
{
    for (int i = blockIdx.x * blockDim.x + threadIdx.x; i < size; i += blockDim.x * gridDim.x)
      c[i] = a[i] + b[i];// Вычислим id нити, в которой происходит суммирование с помощью формулы
}

int SUM(int* c, int size){
    int s = 0;
    for (int i = 0; i < size; i += 1)
      s += c[i];
    return s;
}

int main(int argc, char* argv[])
{
    int grid[] = {8192, 4096};
    int block[] = {1024, 512, 256};
    int vec[] = {1000000, 200000, 40000, 8000, 1000};
 	  for(int i : grid){
        for(int j : block){
                for(int n : vec){
                  int GRID_SIZE = i;

                  int res1=0;
                  int res2=0;

                  int BLOCK_SIZE = j;

                  printf("Vector size = %d\n", n);
                  printf("GridDim = %d\n", GRID_SIZE);
                  printf("BlockDim = %d\n", BLOCK_SIZE);

                  double time1 = 0;
                  double time2 = 0;

                  // Size, in bytes, of each vector
                  int n2b = n * sizeof(int);

                  //BLOCK_SIZE - количество нитей в блоке кратное 32
                  int n2 = n;
                      //BLOCK_NUMBER * BLOCK_SIZE * sizeof(int);;

                  // Выделение памяти на хосте
                  int* a = (int*)calloc(n2, sizeof(int));
                  int* b = (int*)calloc(n2, sizeof(int));
                  int* c = (int*)calloc(n2, sizeof(int));
                  int* c2 = (int*)calloc(n2, sizeof(int));

                  // Инициализация массивов
                  for (int i = 0; i < n; i++) {
                      a[i] = 1;
                      b[i] = 1;
                  }

                  srand(time(0));
		              double begin = clock();
                  for (int i = 0; i < n; i++) {
                      c2[i] = a[i]+b[i];
                  }
                  double end = (clock() - begin) / CLOCKS_PER_SEC;
                  time2 += end;

                  // Выделение памяти на устройстве
                  int* adev = NULL;
                  cudaError_t cuerr = cudaMalloc((void**)&adev, n2b);
                  if (cuerr != cudaSuccess)
                  {
                      fprintf(stderr, "Cannot allocate device array for a: %s\n",
                          cudaGetErrorString(cuerr));
                      return 0;
                  }

                  int* bdev = NULL;
                  cuerr = cudaMalloc((void**)&bdev, n2b);
                  if (cuerr != cudaSuccess)
                  {
                      fprintf(stderr, "Cannot allocate device array for b: %s\n",
                          cudaGetErrorString(cuerr));
                      return 0;
                  }

                  int* cdev = NULL;
                  cuerr = cudaMalloc((void**)&cdev, n2b);
                  if (cuerr != cudaSuccess)
                  {
                      fprintf(stderr, "Cannot allocate device array for c: %s\n",
                          cudaGetErrorString(cuerr));
                      return 0;
                  }

                  // Создание обработчиков событий
                  cudaEvent_t start, stop;
                  float gpuTime = 0.0f;
                  cuerr = cudaEventCreate(&start);
                  if (cuerr != cudaSuccess)
                  {
                      fprintf(stderr, "Cannot create CUDA start event: %s\n",
                          cudaGetErrorString(cuerr));
                      return 0;
                  }

                  cuerr = cudaEventCreate(&stop);
                  if (cuerr != cudaSuccess)
                  {
                      fprintf(stderr, "Cannot create CUDA end event: %s\n",
                          cudaGetErrorString(cuerr));
                      return 0;
                  }

                  // Копирование данных с хоста на девайс
                  cuerr = cudaMemcpy(adev, a, n2b, cudaMemcpyHostToDevice);
                  if (cuerr != cudaSuccess)
                  {
                      fprintf(stderr, "Cannot copy a array from host to device: %s\n",
                          cudaGetErrorString(cuerr));
                      return 0;
                  }

                  cuerr = cudaMemcpy(bdev, b, n2b, cudaMemcpyHostToDevice);
                  if (cuerr != cudaSuccess)
                  {
                      fprintf(stderr, "Cannot copy b array from host to device: %s\n",
                          cudaGetErrorString(cuerr));
                      return 0;
                  }

                  // Установка точки старта
                  cuerr = cudaEventRecord(start, 0);
                  if (cuerr != cudaSuccess)
                  {
                      fprintf(stderr, "Cannot record CUDA event: %s\n",
                          cudaGetErrorString(cuerr));
                      return 0;
                  }

                  //Запуск ядра
                  addKernel << < GRID_SIZE, BLOCK_SIZE >> > (cdev, adev, bdev, n);

                  cuerr = cudaGetLastError();
                  if (cuerr != cudaSuccess)
                  {
                      fprintf(stderr, "Cannot launch CUDA kernel: %s\n",
                          cudaGetErrorString(cuerr));
                      return 0;
                  }

                  // Синхронизация устройств
                  cuerr = cudaDeviceSynchronize();
                  if (cuerr != cudaSuccess)
                  {
                      fprintf(stderr, "Cannot synchronize CUDA kernel: %s\n",
                          cudaGetErrorString(cuerr));
                      return 0;
                  }

                  // Установка точки окончания
                  cuerr = cudaEventRecord(stop, 0);
                  if (cuerr != cudaSuccess)
                  {
                      fprintf(stderr, "Cannot copy c array from device to host: %s\n",
                          cudaGetErrorString(cuerr));
                      return 0;
                  }

                  // Копирование результата на хост
                  cuerr = cudaMemcpy(c, cdev, n2b, cudaMemcpyDeviceToHost);
                  if (cuerr != cudaSuccess)
                  {
                      fprintf(stderr, "Cannot copy c array from device to host: %s\n",
                          cudaGetErrorString(cuerr));
                      return 0;
                  }

                  // Расчет времени
                  cuerr = cudaEventElapsedTime(&gpuTime, start, stop);
                  time1+=gpuTime/1000;
                  cudaEventDestroy(start);
                  cudaEventDestroy(stop);
                  res1=SUM(c,n);
                  res2=SUM(c2,n);

                  // Очищение памяти
                  cudaFree(adev);
                  cudaFree(bdev);
                  cudaFree(cdev);
                  free(a);
                  free(b);
                  free(c);
                  free(c2);

                  std::cout<<"Time on GPU: "<<time1<<std::endl;
                  std::cout<<"Time on CPU: "<<time2<<std::endl;
                  std::cout<<"Acceleration: "<<time2/time1<<std::endl;
                  std::cout<<"Sum on GPU: "<<res1<<std::endl;
                  std::cout<<"Sum on CPU: "<<res2<<std::endl<<std::endl<<std::endl;
                }
        }
    }

    return 0;
}

Vector size = 1000000
GridDim = 8192
BlockDim = 1024
Time on GPU: 0.000274528
Time on CPU: 0.004891
Acceleration: 17.816
Sum on GPU: 2000000
Sum on CPU: 2000000


Vector size = 200000
GridDim = 8192
BlockDim = 1024
Time on GPU: 0.000126016
Time on CPU: 0.00091
Acceleration: 7.2213
Sum on GPU: 400000
Sum on CPU: 400000


Vector size = 40000
GridDim = 8192
BlockDim = 1024
Time on GPU: 0.000122176
Time on CPU: 0.000126
Acceleration: 1.0313
Sum on GPU: 80000
Sum on CPU: 80000


Vector size = 8000
GridDim = 8192
BlockDim = 1024
Time on GPU: 0.000117824
Time on CPU: 2.8e-05
Acceleration: 0.237643
Sum on GPU: 16000
Sum on CPU: 16000


Vector size = 1000
GridDim = 8192
BlockDim = 1024
Time on GPU: 0.000119072
Time on CPU: 4e-06
Acceleration: 0.0335931
Sum on GPU: 2000
Sum on CPU: 2000


Vector size = 1000000
GridDim = 8192
BlockDim = 512
Time on GPU: 9.1904e-05
Time on CPU: 0.004693
Acceleration: 51.0642
Sum on GPU: 2000000
Sum on CPU: 2000000


Vector size = 200000
GridDim = 8192
BlockDim = 5